In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.transforms import Affine2D
import time
from matplotlib.offsetbox import AnchoredText 
import mplcursors 
import os
import json
from gearbox import Gearbox

import itertools
import math

from tqdm import tqdm

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4" # select number of desired GPU to be used
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

## To Move into .py helper file:

variables: z_s, z_p2, z_r2, xs, xp1, xp2, xr2

In [4]:
config = {
  "MODULE": 0.8e-3,
  "MC": 0.8467e-3,
  "MU": 0.21,
  "ALPHA": 20,
  "XS": 0.476,
  "XP1": 0.762,
  "XP2": 0.536,
  "XR1": 2,
  "XR2": 1.21,
  "XS_RANGE": [-0.5, 0.5, 0.25],
  "XP1_RANGE": [-1, 1, 0.25],
  "XP2_RANGE": [-0.5, 0.5, 0.25],
  "XR2_RANGE": [-1.5, 1.5, 0.25],
  "N_PLANETS": 3,
  "SUN_LIMITS": [4, 26],
  "P1_LIMITS": [32, 42],
  "P2_LIMITS": [25, 35],
  "R2_LIMITS": [70,86],
  "STEPS": [2, 1, 1, 1],
  "MIN_RATIO_THRESHOLD": 90,
  "MIN_FORWARD_THRESHOLD": 0.30,
  "MIN_BACKWARD_THRESHOLD": 0.30
}


In [5]:
MODULE = config["MODULE"]
MU = config["MU"]
MC = config.get("MC", MODULE)
ALPHA = np.radians(config.get("ALPHA", 20))
XR1 = config.get("XR1", 0.0)
N_PLANETS = config["N_PLANETS"]
SUN_LIMITS = tuple(config["SUN_LIMITS"])
P1_LIMITS  = tuple(config["P1_LIMITS"])
P2_LIMITS  = tuple(config["P2_LIMITS"])
R2_LIMITS  = tuple(config["R2_LIMITS"])
STEPS = tuple(config["STEPS"])
MIN_RATIO_THRESHOLD = config["MIN_RATIO_THRESHOLD"]
MIN_FORWARD_THRESHOLD = config["MIN_FORWARD_THRESHOLD"]
MIN_BACKWARD_THRESHOLD = config["MIN_BACKWARD_THRESHOLD"]

SLICE_GEAR_RATIO = True 
TARGET_GEAR_RATIO = 100
RATIO_TOLERANCE = 10
DISPLAY_BEST_GEARBOX = False
DISPLAY_ITERATIONS_PLOT = True
RUN_SAMPLE_TRIAL = False
MAX_CANDIDATES = 1000

In [6]:
def generate_options(lower: int, upper: int, step: int = 1):
    return list(range(lower, upper + 1, step))

def generate_range_options(range_vals):
    low, high, step = range_vals
    opts = []
    current = low
    while current <= high + 1e-9:
        opts.append(round(current, 5))
        current += step
    return opts

def generate_all_options(sun_limits: tuple, p1_limits: tuple, p2_limits: tuple, r2_limits: tuple, 
                         steps: tuple = (1,1,1,1)):
    step_sun, step_p1, step_p2, step_r2 = steps
    return {
        "sun": generate_options(sun_limits[0], sun_limits[1], step_sun),
        "p1": generate_options(p1_limits[0], p1_limits[1], step_p1),
        "p2": generate_options(p2_limits[0], p2_limits[1], step_p2),
        "r2": generate_options(r2_limits[0], r2_limits[1], step_r2)
    }

In [7]:
options = generate_all_options(SUN_LIMITS, P1_LIMITS, P2_LIMITS, R2_LIMITS, STEPS)
sun_options = options["sun"]
p1_options  = options["p1"]
p2_options  = options["p2"]
r2_options  = options["r2"]
xs_options = generate_range_options(config["XS_RANGE"])
xp1_options = generate_range_options(config["XP1_RANGE"])
xp2_options = generate_range_options(config["XP2_RANGE"])
xr2_options = generate_range_options(config["XR2_RANGE"])

In [8]:
# manual overwrite to minimize run time.
sun_options = [5, 6, 7, 8, 9, 10] # note, these are z_sh (half z_s)
p2_options = [26, 27, 28, 29, 30, 31, 32]
r2_options = [74, 77, 78, 80, 81, 82, 83] # NOTE - add 71??
xs_options = [0.3, 0.4, 0.5]
xp1_options = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
xp2_options = [-0.5, -0.4, -0.3, 0.3, 0.4, 0.5] # NOTE - add 0.2??
xr2_options = [0.2, 0.3, 0.4, 0.5, 0.8, 0.9, 1, 1.1]

In [9]:
# z_sh                                 7
# z_sh_1                               8
# z_sh_2                               9
# z_p2                                28
# z_p2_1                              31
# z_p2_2                              26
# z_r2                                82
# z_r2_1                              74
# z_r2_2                              83
# x_s                                0.5
# x_s_1                              0.4
# x_s_2                              0.3
# x_p1                               1.0
# x_p1_1                             0.9
# x_p1_2                             0.8
# x_p2                              -0.3
# x_p2_1                            -0.4
# x_p2_2                            -0.5
# x_r2                          0.461185
# x_r2_1                        0.361185
# x_r2_2                        0.261185

In [10]:
# z_sh                                 7
# z_sh_1                               8
# z_sh_2                               5
# z_p2                                29
# z_p2_1                              32
# z_p2_2                              31
# z_r2                                80
# z_r2_1                              82
# z_r2_2                              83
# x_s                                0.5
# x_s_1                              0.4
# x_s_2                              0.3
# x_p1                          0.613076
# x_p1_1                        0.513076
# x_p1_2                        0.413076
# x_p2                               0.5
# x_p2_1                             0.4
# x_p2_2                             0.3
# x_r2                          1.077693
# x_r2_1                        0.977693
# x_r2_2                        0.877693

In [ ]:
# best_so_far =  {'z_sh': 7, 'z_sh_1': 10, 'z_sh_2': 5, 'z_p2': 30, 'z_p2_1': 32, 'z_p2_2': 28, 'z_r2': 80, 'z_r2_1': 74, 'z_r2_2': 82, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 1.0, 'x_p1_1': 0.9, 'x_p1_2': 0.8, 'x_p2': -0.3, 'x_p2_1': -0.4, 'x_p2_2': -0.5, 'x_r2': 0.477546, 'x_r2_1': 0.377546, 'x_r2_2': 0.277546}
# b_guess = list_to_param(
#     ((5, 7, 8)[::-1],
#  (27, 30, 31)[::-1],best_so_far =  {'z_sh': 7, 'z_sh_1': 10, 'z_sh_2': 5, 'z_p2': 30, 'z_p2_1': 32, 'z_p2_2': 28, 'z_r2': 80, 'z_r2_1': 74, 'z_r2_2': 82, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 1.0, 'x_p1_1': 0.9, 'x_p1_2': 0.8, 'x_p2': -0.3, 'x_p2_1': -0.4, 'x_p2_2': -0.5, 'x_r2': 0.477546, 'x_r2_1': 0.377546, 'x_r2_2': 0.277546}
# b_guess = list_to_param(
#     ((5, 7, 8)[::-1],
#  (27, 30, 31)[::-1],
#  (71, 78, 80)[::-1],
#  (0.3, 0.4, 0.5)[::-1],
#  (0.5, 0.6, 0.7)[::-1],
#  (0.2, 0.3, 0.4)[::-1],
#  (0.9, 1, 1.1)[::-1])
# )
# c_guess = {'z_sh': 8, 'z_sh_1': 7, 'z_sh_2': 5, 'z_p2': 31, 'z_p2_1': 29, 'z_p2_2': 28, 'z_r2': 80, 'z_r2_1': 78, 'z_r2_2': 71, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.68612, 'x_p1_1': 0.562688, 'x_p1_2': 0.462688, 'x_p2': 0.434784, 'x_p2_1': 0.334784, 'x_p2_2': 0.228267, 'x_r2': 1.059951, 'x_r2_1': 0.959951, 'x_r2_2': 0.859951}
# new_guess = list_to_param((np.array((14, 12, 10))/2,
#  (27, 30, 31),
#  (77, 78, 80),
#  (0.3, 0.4, 0.5)[::-1],
#  (0.6, 0.7, 0.8)[::-1],
#  (0.2, 0.3, 0.4)[::-1],
#  (0.8, 0.9, 1)[::-1]))
#  (71, 78, 80)[::-1],
#  (0.3, 0.4, 0.5)[::-1],
#  (0.5, 0.6, 0.7)[::-1],
#  (0.2, 0.3, 0.4)[::-1],
#  (0.9, 1, 1.1)[::-1])
# )
# c_guess = {'z_sh': 8, 'z_sh_1': 7, 'z_sh_2': 5, 'z_p2': 31, 'z_p2_1': 29, 'z_p2_2': 28, 'z_r2': 80, 'z_r2_1': 78, 'z_r2_2': 71, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.68612, 'x_p1_1': 0.562688, 'x_p1_2': 0.462688, 'x_p2': 0.434784, 'x_p2_1': 0.334784, 'x_p2_2': 0.228267, 'x_r2': 1.059951, 'x_r2_1': 0.959951, 'x_r2_2': 0.859951}
# new_guess = list_to_param((np.array((14, 12, 10))/2,
#  (27, 30, 31),
#  (77, 78, 80),
#  (0.3, 0.4, 0.5)[::-1],
#  (0.6, 0.7, 0.8)[::-1],
#  (0.2, 0.3, 0.4)[::-1],
#  (0.8, 0.9, 1)[::-1]))

# Main

In [11]:
#generate all possible subsets
num_per_param = 3

all_params = [sun_options, p2_options, r2_options, xs_options, xp1_options, xp2_options, xr2_options]

ss_vals = []

for param in all_params:
    ss_vals.append(list(itertools.combinations(param, num_per_param)))

test_len = total_combinations = math.prod(len(subsets) for subsets in ss_vals)

# print(f'Total designs to test: {len(list(itertools.product(*ss_vals)))}')
print(f'Total designs to test: {test_len}')

Total designs to test: 960400000


In [12]:
import sys

start_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from utils import score_vals

In [13]:
# score the subsets, keep the highest score
high_score = 0
best_vals = []
# counter = 0
for combination_of_vals in itertools.product(*ss_vals):
    score = score_vals(list(combination_of_vals))
    if score > high_score:
        best_vals = combination_of_vals
        high_score = score
    # counter+=1
    # if counter > 1000:
    #     break

/home/gmc/Python/Mod_DOE/utils.py:56: RuntimeWarning: divide by zero encountered in divide
  ratios = 1 / gr_s
/home/gmc/Python/Mod_DOE/utils.py:82: RuntimeWarning: divide by zero encountered in divide
  ratios = 1 / gr_s


KeyboardInterrupt: 

In [14]:
best_vals

((5, 8, 9),
 (26, 30, 31),
 (78, 80, 83),
 (0.3, 0.4, 0.5),
 (0.7, 0.8, 0.9),
 (-0.5, -0.4, -0.3),
 (0.2, 0.3, 0.4))

In [15]:
high_score

164